In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import scipy as sc

In [196]:
class Station():
    def __init__(self, station_name, num_bikes, num_docks, arr_stream):
        
        #Initialization of Station Class:
        #station_name: Name of station (string)
        #num_bikes: number of bikes at the station initially (int)
        #num_docks: number of docks at the station initially (int)
        #arr_stream: data stream containing our arrivals to the station (Pandas DataFrame)
        
        self.station_name = station_name
        self.num_bikes = num_bikes
        self.num_docks = num_docks
        self.arr_stream = arr_stream
        self.simulate(num_bikes, num_docks, arr_stream)
        print("Station " + station_name + " has been initialized")
        
    def simulate(self, num_bikes, num_docks, arr_stream):
        
        ##Initialization Information:##
        
        num_bikes = num_bikes #initial number of bikes
        num_docks = num_docks #initial number of docks
        arr_stream = arr_stream #import arrival stream
        arr_stream["Cust_ID"] = np.arange(len(arr_stream)) #Create customer ID for each customer
        
        ##Initialize queues and arrays to store information##
        
        #Bike_Array: Keeps track of the number of bikes
        num_bike_arr = []        
        #Dock_Array: Keeps track of the number of docks
        num_dock_arr = []
        
        #Borrowing_Queue
        borrowing_queue_length_arr = [] #Array that keeps track of the length of the borrowing queue        
        borrowing_queue_length = 0 #Length of the borrowing queue
        borrowing_queue_customer_ID = [] #keeps track of borrowing customers who join the borrowing queue
        borrowing_queue_customer_in_time = [] #keeps track of the time borrowing customers join the borrowing queue
        borrowing_queue_customer_out_time = [] #keeps track of the time borrowing customers exit the borrowing queue
        
        #Returning_Queue
        returning_queue_length_arr = [] #Array that keeps track of the length of the borrowing queue        
        returning_queue_length = 0 #Length of the borrowing queue
        returning_queue_customer_ID = [] #keeps track of borrowing customers who join the borrowing queue
        returning_queue_customer_in_time = [] #keeps track of the time borrowing customers join the borrowing queue
        returning_queue_customer_out_time = [] #keeps track of the time borrowing customers exit the borrowing queue
        
        #Instant_Service
        instant_service_customer_ID = [] #Keeps track of the customers who get instant service
        
        #Add Initial Information#
        num_bike_arr.append(num_bikes)
        num_dock_arr.append(num_docks)
        borrowing_queue_length_arr.append(borrowing_queue_length)
        returning_queue_length_arr.append(returning_queue_length)
        
        for i in range(len(arr_stream)): #For all the values in our data stream
            if i == 0: #For first customer
                
                if arr_stream.iloc[0, :]["Type_of_Customer"] == "Borrowing": #If the customer is borrowing bikes
                    
                    if num_bikes == 0: #If the no bikes to borrow
                        #Number of bikes and docks remain the same
                        
                        print("--------------------------------")
                        print("Iteration " + str(i))
                        print("Scenario 1")
                        print("Num_Bikes: " + str(num_bikes))
                        print("Num_Docks: " + str(num_docks))
                            
                        borrowing_queue_customer_ID.append(arr_stream.iloc[0, :]["Cust_ID"]) #Add ID to borrowing queue
                        borrowing_queue_customer_in_time.append(arr_stream.iloc[0, :]["Arrival_Time"]) #Add Arrival_Time
                        borrowing_queue_length += 1 #borrowing queue length increases by 1
                        borrowing_queue_length_arr.append(borrowing_queue_length) #append borrowing queue length to array
                    else: #There are bikes to borrow
                        
                        print("--------------------------------")
                        print("Iteration " + str(i))
                        print("Scenario 2")
                        print("Num_Bikes: " + str(num_bikes))
                        print("Num_Docks: " + str(num_docks))
                        
                        instant_service_customer_ID.append(arr_stream.iloc[0, :]["Cust_ID"]) #Append Cust_ID to instant service array
                        num_bikes = num_bikes - 1 #Customer borrows bike (num_bike goes down by 1)
                        num_docks = num_docks + 1 #Customer borrows bike (num_dock goes up by 1)
                        
                        returning_queue_length_arr.append(returning_queue_length) #append returning queue length to array
                        borrowing_queue_length_arr.append(borrowing_queue_length)
                        num_bike_arr.append(num_bikes) #Append number of bikes to array
                        num_dock_arr.append(num_docks) #Append number of docks to array
                        
                else: #If the customer is returning bikes
                    if num_docks == 0: #If there are no docks to return
                        
                        print("--------------------------------")
                        print("Iteration " + str(i))
                        print("Scenario 3")
                        print("Num_Bikes: " + str(num_bikes))
                        print("Num_Docks: " + str(num_docks))
                        
                        returning_queue_customer_ID.append(arr_stream.iloc[0, :]["Cust_ID"]) #Add ID to returning queue
                        returning_queue_customer_in_time.append(arr_stream.iloc[0, :]["Arrival_Time"]) #Add Arrival_Time
                        returning_queue_length = 1 #returning queue length increases by 1
                        
                        
                        num_bike_arr.append(num_bikes)
                        num_dock_arr.append(num_docks)
                        returning_queue_length_arr.append(returning_queue_length) #append returning queue length to array
                        borrowing_queue_length_arr.append(borrowing_queue_length)
                    else: #There are docks available
                        
                        print("--------------------------------")
                        print("Iteration " + str(i))
                        print("Scenario 4")
                        print("Num_Bikes: " + str(num_bikes))
                        print("Num_Docks: " + str(num_docks))
                        
                        instant_service_customer_ID.append(arr_stream.iloc[0, :]["Cust_ID"]) #Append Cust_ID to instant service array
                        num_bikes = num_bikes + 1 #Customer returns bike (num_bike goes up by 1)
                        num_docks = num_docks - 1 #Customer returns bike (num_dock goes down by 1)
                        num_bike_arr.append(num_bikes) #Append number of bikes to array
                        num_dock_arr.append(num_docks) #Append number of docks to array
                        
            else: #For remaining customers
                
                if arr_stream.iloc[i, :]["Type_of_Customer"] == "Borrowing": #If customer is borrowing
                    
                    if borrowing_queue_length > 0: #Borrowing queue has people
                        #Try to serve queued customers first
                        
                        if returning_queue_length > 0: #If returning queue has people
                            
                            print("--------------------------------")
                            print("Iteration " + str(i))
                            print("Scenario 5")
                            print("Num_Bikes: " + str(num_bikes))
                            print("Num_Docks: " + str(num_docks))
                            
                            returning_queue_length -= 1 #Decrease the returning queue individual by one
                            borrowing_queue_length -= 1 #Decrease the borrowing queue individuals by one
                            returning_queue_length_arr.append(returning_queue_length) #Append queue length                            
                            returning_queue_customer_out_time.append(arr_stream.iloc[i, :]["Arrival_Time"]) #Update exit time for returning_queue customer
                            borrowing_queue_customer_out_time.append(arr_stream.iloc[i, :]["Arrival_Time"]) #Update exit time for borrowing_queue customer
                            
                            num_bike_arr.append(num_bikes) #Append number of bikes to array
                            num_dock_arr.append(num_docks) #Append number of docks to array
                            
                            #Add current customer to borrowing_queue
                            borrowing_queue_customer_ID.append(arr_stream.iloc[i, :]["Cust_ID"]) #Add ID to borrowing queue
                            borrowing_queue_customer_in_time.append(arr_stream.iloc[i, :]["Arrival_Time"]) #Add Arrival_Time
                            borrowing_queue_length += 1 #borrowing queue length increases by 1
                            borrowing_queue_length_arr.append(borrowing_queue_length) #append length to borrowing_queue
                            
                        elif returning_queue_length == 0: #returning queue has no people
                            
                            print("--------------------------------")
                            print("Iteration " + str(i))
                            print("Scenario 6")
                            print("Num_Bikes: " + str(num_bikes))
                            print("Num_Docks: " + str(num_docks))
                            
                            borrowing_queue_customer_ID.append(arr_stream.iloc[i, :]["Cust_ID"])#Add customer ID to borrowing queue
                            borrowing_queue_customer_in_time.append(arr_stream.iloc[i, :]["Arrival_Time"]) #Add Arrival_Time
                            borrowing_queue_length += 1 #returning queue length increases by 1
                            borrowing_queue_length_arr.append(borrowing_queue_length) #append returning queue length to array
                            
                    elif borrowing_queue_length == 0 and num_bikes > 0: #If there is no borrowing queue and there are bikes available
                        #Instant borrowing of bike
                        
                        #num_bikes = num_bikes - 1 #Customer borrows bike (num_bike goes down by 1)
                        #num_docks = num_docks + 1 #Customer borrows bike (num_dock goes up by 1)
                            
                        #num_bike_arr.append(num_bikes) #Append number of bikes to array
                        #num_dock_arr.append(num_docks) #Append number of docks to array
                        
                            
                        if returning_queue_length > 0: #If there is someone in the returning queue
                            #Instant service opens up a new dock, allow returning customer to end wait
                            
                            print("--------------------------------")
                            print("Iteration " + str(i))
                            print("Scenario 7")
                            print("Num_Bikes: " + str(num_bikes))
                            print("Num_Docks: " + str(num_docks))
                            
                            returning_queue_customer_out_time.append(arr_stream.iloc[i, :]["Arrival_Time"])
                            returning_queue_length -= 1
                            
                            num_bike_arr.append(num_bikes)
                            num_dock_arr.append(num_docks)
                            returning_queue_length_arr.append(returning_queue_length)
                            borrowing_queue_length_arr.append(borrowing_queue_length)
                            
                        else: #There is nobody in the returning queue ()
                            
                            print("--------------------------------")
                            print("Iteration " + str(i))
                            print("Scenario 8")
                            print("Num_Bikes: " + str(num_bikes))
                            print("Num_Docks: " + str(num_docks))
                            instant_service_customer_ID.append(arr_stream.iloc[i,:]["Cust_ID"]) #Append Cust_ID to instant service array
                            
                            
                            num_bikes = num_bikes - 1 #Customer borrows bike (num_bike goes down by 1)
                            num_docks = num_docks + 1 #Customer borrows bike (num_dock goes up by 1)
                            num_bike_arr.append(num_bikes) #Append number of bikes to array
                            num_dock_arr.append(num_docks) #Append number of docks to array
                            
                            returning_queue_length_arr.append(returning_queue_length)
                            borrowing_queue_length_arr.append(borrowing_queue_length)
                            
                    elif borrowing_queue_length == 0 and num_bikes == 0: #No bikes to borrow
                        
                        print("--------------------------------")
                        print("Iteration " + str(i))
                        print("Scenario 9")
                        print("Num_Bikes: " + str(num_bikes))
                        print("Num_Docks: " + str(num_docks))
                        
                        #Add to borrowing queue
                        borrowing_queue_customer_ID.append(arr_stream.iloc[i, :]["Cust_ID"])#Add customer ID to borrowing queue
                        borrowing_queue_customer_in_time.append(arr_stream.iloc[i, :]["Arrival_Time"]) #Add Arrival_Time
                    
                        
                else: #If customer is returning
                    
                    
                    if returning_queue_length > 0: #Returning queue has people
                    #Try to service queued customers first
                    
                        if borrowing_queue_length > 0: #If returning queue has people
                            
                            print("--------------------------------")
                            print("Iteration " + str(i))
                            print("Scenario 10")
                            print("Num_Bikes: " + str(num_bikes))
                            print("Num_Docks: " + str(num_docks))                            
                            returning_queue_length -= 1 #Decrease the returning queue individual by one
                            borrowing_queue_length -= 1 #Decrease the borrowing queue individual by one
                            borrowing_queue_length_arr.append(borrowing_queue_length) #Append queue length                            
                            returning_queue_customer_out_time.append(arr_stream.iloc[i, :]["Arrival_Time"]) #Update exit time for returning_queue customer
                            borrowing_queue_customer_out_time.append(arr_stream.iloc[i, :]["Arrival_Time"]) #Update exit time for borrowing_queue customer
                                                                                
                            num_bike_arr.append(num_bikes) #Append number of bikes to array
                            num_dock_arr.append(num_docks) #Append number of docks to array

                            #Add current customer to returning_queue
                            returning_queue_customer_ID.append(arr_stream.iloc[i, :]["Cust_ID"]) #Add ID to borrowing queue
                            returning_queue_customer_in_time.append(arr_stream.iloc[i, :]["Arrival_Time"]) #Add Arrival_Time
                            returning_queue_length += 1 #returning queue length increases by 1
                            returning_queue_length_arr.append(returning_queue_length) #append length to returning_queue

                        elif borrowing_queue_length == 0: #borrowing queue has no people
                            #Current Add
                            
                            print("--------------------------------")
                            print("Iteration " + str(i))
                            print("Scenario 11")
                            print("Num_Bikes: " + str(num_bikes))
                            print("Num_Docks: " + str(num_docks))
                            
                            returning_queue_customer_ID.append(arr_stream.iloc[i, :]["Cust_ID"])#Add customer ID to returning queue
                            returning_queue_customer_in_time.append(arr_stream.iloc[i, :]["Arrival_Time"]) #Add Arrival_Time
                            returning_queue_length += 1 #returning queue length increases by 1
                            
                            
                            num_bike_arr.append(num_bikes)
                            num_dock_arr.append(num_docks)
                            returning_queue_length_arr.append(returning_queue_length) #append returning queue length to array
                            borrowing_queue_length_arr.append(borrowing_queue_length)
                            
                    elif returning_queue_length == 0 and num_docks > 0: #If there is no returning queue and there are bikes available
                        #Instant_Service
                        
                                              
                        #num_bikes = num_bikes + 1 #Customer returns bike (num_bike goes up by 1)
                        #num_docks = num_docks - 1 #Customer borrows bike (num_dock goes down by 1)
                        #num_bike_arr.append(num_bikes) #Append number of bikes to array
                        #num_dock_arr.append(num_docks) #Append number of docks to array
                        
                        if borrowing_queue_length > 0: #If there is someone in the returning queue
                            #
                            
                            print("--------------------------------")
                            print("Iteration " + str(i))
                            print("Scenario 12")
                            print("Num_Bikes: " + str(num_bikes))
                            print("Num_Docks: " + str(num_docks))
                            
                            borrowing_queue_customer_out_time.append(arr_stream.iloc[i, :]["Arrival_Time"])
                            borrowing_queue_length -= 1
                            
                            borrowing_queue_length_arr.append(borrowing_queue_length)
                            returning_queue_length_arr.append(returning_queue_length)
                            num_bike_arr.append(num_bikes) #Append number of bikes to array
                            num_dock_arr.append(num_docks) #Append number of docks to array
                            
                        else: #There is nobody in the returning queue ()

                            print("--------------------------------")
                            print("Iteration " + str(i))
                            print("Scenario 13")
                            print("Num_Bikes: " + str(num_bikes))
                            print("Num_Docks: " + str(num_docks))
                        
                            instant_service_customer_ID.append(arr_stream.iloc[i, :]["Cust_ID"]) #Append Cust_ID to instant service array
                            
                            
                            num_bikes = num_bikes + 1 #Customer borrows bike (num_bike goes down by 1)
                            num_docks = num_docks - 1 #Customer borrows bike (num_dock goes up by 1)
                            
                            num_bike_arr.append(num_bikes) #Append number of bikes to array
                            num_dock_arr.append(num_docks) #Append number of docks to array
                            returning_queue_length_arr.append(returning_queue_length)
                            borrowing_queue_length_arr.append(borrowing_queue_length)
                            
                    elif returning_queue_length == 0 and num_docks == 0: #No docks to return bikes to
                        #Add to returning queue 
                        print("--------------------------------")
                        print("Iteration " + str(i))
                        print("Scenario 14")
                        print("Num_Bikes: " + str(num_bikes))
                        print("Num_Docks: " + str(num_docks))
                        
                        returning_queue_customer_ID.append(arr_stream.iloc[i, :]["Cust_ID"])#Add customer ID to borrowing queue
                        returning_queue_customer_in_time.append(arr_stream.iloc[i, :]["Arrival_Time"]) #Add Arrival_Time
                        returning_queue_length += 1
                        
                        borrowing_queue_length_arr.append(borrowing_queue_length)
                        returning_queue_length_arr.append(returning_queue_length)
                        num_bike_arr.append(num_bikes)
                        num_dock_arr.append(num_docks)
             
        self.num_bike_arr = num_bike_arr
        self.num_dock_arr = num_dock_arr
        self.borrowing_queue_length_arr = borrowing_queue_length_arr
        self.returning_queue_length_arr = returning_queue_length_arr
        
        self.borrowing_queue_customer_ID = borrowing_queue_customer_ID
        self.borrowing_queue_customer_in_time = borrowing_queue_customer_in_time
        self.borrowing_queue_customer_out_time = borrowing_queue_customer_out_time
        
        self.returning_queue_customer_ID = returning_queue_customer_ID
        self.returning_queue_customer_in_time = returning_queue_customer_in_time
        self.returning_queue_customer_out_time = returning_queue_customer_out_time
        
        self.instant_service_customer_ID = instant_service_customer_ID
        
    def get_num_bike_arr(self):
        return self.num_bike_arr
    
    def get_num_dock_arr(self):
        return self.num_dock_arr
    
    def get_borrowing_queue_lengths(self):
        return self.borrowing_queue_length_arr
    
    def get_returning_queue_lengths(self):
        return self.returning_queue_length_arr
    
    def get_borrowing_queue(self):
        #If in time is less than out time, add 
        
        len_diff = len(self.borrowing_queue_customer_in_time) - len(self.borrowing_queue_customer_out_time)
        
        for i in np.arange(len_diff):
            self.borrowing_queue_customer_out_time.append("Not Served")
        borrowing_df = pd.DataFrame({"Cust_ID": self.borrowing_queue_customer_ID,
                     "Cust_In_Time": self.borrowing_queue_customer_in_time,
                     "Cust_Out_Time": self.borrowing_queue_customer_out_time})        
        return borrowing_df
    
    def get_returning_queue(self):
        #If in time is less than out time, add 
        len_diff = len(self.returning_queue_customer_in_time) - len(self.returning_queue_customer_out_time)
        for i in np.arange(len_diff):
            self.returning_queue_customer_out_time.append("Not Served")
            
        returning_df = pd.DataFrame({"Cust_ID": self.returning_queue_customer_ID,
                                    "Cust_In_Time": self.returning_queue_customer_in_time,
                                    "Cust_Out_Time": self.returning_queue_customer_out_time})        
        return returning_df
    
    def get_instant_service_customer_queue(self):
        return pd.DataFrame({"Cust_ID": self.instant_service_customer_ID}) #Keeps track of the customers who get instant service


In [220]:
test_df_2 =  pd.DataFrame({"Arrival_Time": [0, 1, 2, 3],
                        "Type_of_Customer": ["Returning", "Borrowing", "Returning", "Borrowing"],
                        "Station_From": ["Joe", "Joe", "Joe", "Joe"]
                       })
test_df_2



,Arrival_Time,Type_of_Customer,Station_From
0,0,Returning,Joe
1,1,Borrowing,Joe
2,2,Returning,Joe
3,3,Borrowing,Joe


In [221]:
test_station = Station("Joe", 5, 0, test_df_2)

--------------------------------
Iteration 0
Scenario 3
Num_Bikes: 5
Num_Docks: 0
--------------------------------
Iteration 1
Scenario 7
Num_Bikes: 5
Num_Docks: 0
--------------------------------
Iteration 2
Scenario 14
Num_Bikes: 5
Num_Docks: 0
--------------------------------
Iteration 3
Scenario 7
Num_Bikes: 5
Num_Docks: 0
Station Joe has been initialized


In [222]:
test_station.get_returning_queue()

,Cust_ID,Cust_In_Time,Cust_Out_Time
0,0,0,1
1,2,2,3


In [223]:
test_station.get_borrowing_queue()

,Cust_ID,Cust_In_Time,Cust_Out_Time


In [224]:
test_station.get_returning_queue_lengths()

[0, 1, 0, 1, 0]

In [225]:
test_station.get_borrowing_queue_lengths()

[0, 0, 0, 0, 0]

In [226]:
test_station.get_instant_service_customer_queue()

,Cust_ID


In [227]:
test_station.get_num_bike_arr()

[5, 5, 5, 5, 5]

In [228]:
test_station.get_num_dock_arr()

[0, 0, 0, 0, 0]